In [ ]:
from zipfile import ZipFile
file_name='/content/drive/My Drive/Severstal-Steel-Defect-detection.zip'
with ZipFile(file_name,'r') as Zip:
  Zip.extractall()

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install tensorflow-addons

In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sb
import os
import tensorflow as tf
import IPython.display as display
from PIL import Image
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Lambda, Flatten,Conv2D,Dense,MaxPooling2D,Dropout,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard
import tensorflow_addons as tfa
import datetime
from tqdm import tqdm
from tensorflow.keras.applications.xception import Xception
from sklearn.model_selection import train_test_split
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Severstal steel defect detection/train.csv/train.csv')
print(df.shape)
df.head(2)

(7095, 3)


,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...


In [ ]:
defect_img=df.ImageId.unique().tolist()

In [ ]:
bin_data=[]
for i in tqdm(os.listdir('/content/drive/MyDrive/Severstal steel defect detection/train_images')):
  if i not in bin_data:#Not to duplicate image
    if i in defect_img:
      bin_data.append(['train_images/'+i,1])
    else:
      bin_data.append(['train_images/'+i,0])

100%|██████████| 12568/12568 [00:02<00:00, 5178.41it/s]


In [ ]:
bin_label=pd.DataFrame(list(map(lambda x:x[0],bin_data)),columns=['ImageId'])
has_defect=pd.DataFrame(list(map(lambda x:x[1],bin_data)),columns=['has_defect'])

In [ ]:
binary_label_pd=pd.concat([bin_label,has_defect],axis=1)

In [ ]:
print('shape : ',binary_label_pd.shape)
binary_label_pd.head(4)

shape :  (12568, 2)


,ImageId,has_defect
0,train_images/e8b114d2a.jpg,0
1,train_images/e98824ca8.jpg,1
2,train_images/e7ced5b76.jpg,1
3,train_images/e857c7372.jpg,1


In [ ]:
X_train, X_test = train_test_split(binary_label_pd, test_size = 0.12, stratify = binary_label_pd.has_defect,random_state=42)
X_train, X_val = train_test_split(X_train, test_size = 0.12, stratify = X_train.has_defect,random_state=42)
print('train shape : ',X_train.shape)
print('test shape : ', X_test.shape)
print('val shape : ', X_val.shape)

train shape :  (9731, 2)
test shape :  (1509, 2)
val shape :  (1328, 2)


In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) # calculates number of true positives
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))      # calculates number of actual positives
    recall = true_positives / (possible_positives + K.epsilon())   # K.epsilon takes care of non-zero divisions
    return recall
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))  #
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from random import random

In [ ]:
def augment(image, label):
    a=random()
    if a>0.2 and a<0.3:
      image = tf.image.random_flip_left_right(image)
      image = tf.clip_by_value(image, 0.0, 1.0)
      return image, label
    if a>0.3 and a<0.4:
      image = tf.image.random_saturation(image, lower=0.4, upper=1.4)
      image = tf.clip_by_value(image, 0.0, 1.0)
      return image, label
    if a>0.4 and a<0.45:
      image = plt.imshow(tf.image.random_brightness(image, max_delta=0.14))
      image = tf.clip_by_value(image, 0.0, 1.0)
      return image, label
    if a>0.45 and a<0.6:
      image=tf.image.random_flip_up_down(image)
      image = tf.clip_by_value(image, 0.0, 1.0)
      return image, label
    if a>0.65 and a<0.75:
      plt.imshow(tf.image.rot90(image))
      image = tf.clip_by_value(image, 0.0, 1.0)
      return image, label
      
    else:
      image = tf.clip_by_value(image, 0.0, 1.0)
      return image, label  

In [ ]:
def preprocess_image(filename, label):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [299,299])
    return image, label

In [ ]:
def tfdata_train(filenames,label,batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((filenames, label))
  dataset = dataset.shuffle(len(filenames))
  dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.map(augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset=dataset.repeat()
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return dataset

In [ ]:
def tfdata_test(filenames,label,batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((filenames, label))
  dataset = dataset.shuffle(len(filenames))
  dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset=dataset.repeat()
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return dataset

In [ ]:
X_train_bin=tfdata_train(X_train.ImageId.tolist(),X_train.has_defect.tolist(),24)
X_val_bin=tfdata_test(X_val.ImageId.tolist(),X_val.has_defect.tolist(),24)

In [ ]:
X_train_bin,X_val_bin,

(<PrefetchDataset shapes: ((None, 299, 299, 3), (None,)), types: (tf.float32, tf.int32)>,
 <PrefetchDataset shapes: ((None, 299, 299, 3), (None,)), types: (tf.float32, tf.int32)>)

In [ ]:
res=Xception(input_shape=(299,299,3),include_top=False)

In [ ]:
gap = GlobalAveragePooling2D()(res.output)
d1=Dense(998,activation='relu',kernel_initializer='he_uniform')(gap)
bn1 = BatchNormalization()(d1)
dp1 = Dropout(0.1)(bn1)
d2=Dense(488,activation='relu',kernel_initializer='he_uniform')(dp1)
bn2 = BatchNormalization()(d2)
dp2 = Dropout(0.1)(bn2)
d3=Dense(48,activation='relu',kernel_initializer='he_uniform')(dp2)
out=Dense(1,activation='sigmoid')(d3)
model1=Model(inputs=res.input,outputs=out)

In [ ]:
model1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',precision_m,recall_m,f1_m])

In [ ]:
filepath="/content/drive/My Drive/binary_model/"+"weights-{epoch:02d}-{val_f1_m:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_f1_m',verbose=1, save_best_only=True, mode='max')
log_dir="/content/drive/My Drive/binary_model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=0, write_graph=True,write_grads=False)
po=[checkpoint,tensorboard_callback]

In [ ]:
model1.fit(X_train_bin,epochs=24,steps_per_epoch=580,validation_data=X_val_bin,validation_steps=120,callbacks=po)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/24


NotFoundError: ignored